# Camping Calculator

The worst possible way to get camping prices in France:
- Download the HTML of the search results on https://campingfrance.com/
- Find the camping sites (totally robust criterion: they're written in all caps)
- Download the camping site detail page HTML
- Find table with prices and div containing GPS coordinates
- Parse text with regexes (they're horribly inconsistent, thus many corner cases)
- Dump info into a dictionary
- Calculate prices for our trip and show summary (if possible)

### Packages

In [1]:
import re
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML, display

### Base Data

Hard-coded for now, but no big deal to change that.

In [2]:
PEOPLE = {
    "Ro": 42,
    "Ev": 35,
    "Ja": 13,
    "Da": 11,
    "To": 7,
    "Jo": 4,
    "Ho": 26,
}

N_CARS = 2
N_PLACES = 2
N_NIGHTS = 7

### Convenience Functions to Pull Stuff out of Dictionaries

In [3]:
def _get(people, age_min, age_max, exclude):
    for name, age in people.items():
        if age_min <= age <= age_max and name not in exclude:
            return name
    else:
        return None


def get_adult(people, age_min, exclude):
    return _get(people, age_min, float("inf"), exclude)


def get_child(people, age_min, exclude):
    return _get(people, age_min, age_max, exclude)


def get_toddler(people, age_max, exclude):
    return _get(people, 0, age_max, exclude)


def _count(people, age_min, age_max, exclude):
    return sum(1 if age_min <= age <= age_max and name not in exclude else 0 for name, age in people.items())


def count_adults(people, age_min, exclude):
    return _count(people, age_min, float("inf"), exclude)


def count_children(people, age_min, age_max, exclude):
    return _count(people, age_min, age_max, exclude)


def count_toddlers(people, age_max, exclude):
    return _count(people, 0, age_max, exclude)

### Find Camping Sites on all Search Result Pages

In [4]:
CAMPINGFRANCE_SEARCH_URL_FMT = (  # hard-coded location :-(
    r"https://www.campingfrance.com/uk"
    "/find-your-campsite"
    "/provence-alpes-cote-dazur"
    "/hautes-alpes"
    "/risoul"
    "{page_suffix}"
)
EXCLUDE_TITLES = ["FFCC", "LPBC", "OT"]

page_suffices = [""]
page_suffices.extend("/page-{}".format(i) for i in range(2, 4))  # hard-coded :-(
campings = {}


def get_campings(soup):
    links = list(soup.find_all("a"))
    campings = {}
    for link in links:
        title = link.get("title")
        if title is None or title != title.upper() or title in EXCLUDE_TITLES:
            continue
        campings[title] = link.get("href")
        
    return campings


for suffix in page_suffices:
    url = CAMPINGFRANCE_SEARCH_URL_FMT.format(page_suffix=suffix)
    response = requests.get(url)
    soup = BeautifulSoup(response.content)
    campings.update(get_campings(soup))


# Turn relative links to full URLs and drop absolute links (ads)
campings = {
    title: {"href": "https://www.campingfrance.com" + href.replace("/uk", "/fr")}
    for title, href in campings.items() if href.startswith("/")
}

### Functions for Finding and Parsing Tables

In [5]:
def get_table_texts(soup):
    tables = soup.find_all("table")
    table_texts = []
    for table in tables:
        table_texts.extend(child.text.strip() for child in table.children)
    return table_texts
    

def get_price(texts, start):
    # Regex for getting the price out of a string like
    # "Flat : 5.50€/day".
    #
    # This regex took quite a while to get right. It needs to handle both
    # integer and floating point prices ("5€" and "5.50€") but still capture
    # the whole thing as a group.
    # Also the begin must be handled, but not too greedily (only non-digits).
    # (Note also that the price is sometimes written as "5,50€/day", sometimes
    # as "5.50€/")
    price_regex = re.compile(r"\D*(\d+(?:,\d{2})*)€.*")

    for text in texts:
        if text.startswith(start):
            match = price_regex.match(text)
            if match is None or match.groups is None:
                continue
            assert len(match.groups()) == 1
            price = match.groups()[0].replace(",", ".")
            return float(price)
    else:
        raise ValueError(
            "no price found in {} for start {!r}".format(texts, start)
        )


def get_age_price(texts, start):
    # Regex for getting both age and price out of a string like
    # "Children (10 years old) : 3.50€/day".
    #
    # Apart from all the troubles of the regex in `get_price`, here we also have
    # the inconsistency of "(10 years old)" vs. "(-10 years old)".
    # TODO: we even have "(3 à 12 years old)", so we need ranges :-(
    age_price_regex = re.compile(r"\D*\(\D*(\d+).*years.*\)\D*(\d+(?:,\d{2})*)€.*")

    for text in texts:
        if text.startswith(start):
            match = age_price_regex.match(text)
            if match is None or match.groups is None:
                continue
            assert len(match.groups()) == 2
            age, price = match.groups()
            return int(age), float(price.replace(",", "."))
    else:
        raise ValueError(
            "no age and price found in {} for start {!r}".format(texts, start)
        )

### Find the GPS Coordinates in a `<div>` Tag

In [6]:
def get_coords(soup):
    coord_regex = re.compile(r'\{"lat":(\d+.\d+),"lng":(\d+.\d+)\}')

    map_divs = soup.find_all("div", attrs={"class": "map"})
    if not map_divs:
        print("no map div found")
        return None, None
    
    for map_div in map_divs:
        if "data-map" in map_div.attrs:
            coord_str = map_div.attrs["data-map"]
            break
    else:
        print("no 'data-map' attribute in {}".format(map_divs))
        return None, None

    match = coord_regex.match(coord_str)
    if match is None:
        print("no regex match in {!r}".format(coord_str))
        return None, None
    
    if match.groups is None or len(match.groups()) != 2:
        print("lat/long not found in in {!r}".format(coord_str))
        return None, None
    
    lat, lng = map(float, match.groups())
    return lat, lng

### Assemble all Information in a Dictionary

In [7]:
for site in campings:
    # Title case makes it read better
    campings[site]["name"] = site.title()

    # Download page and parse HTML
    response = requests.get(campings[site]["href"])
    assert response.ok
    soup = BeautifulSoup(response.content)

    # Find tables containing rates and get prices
    has_rates = bool(soup.find_all("a", attrs={"title": "Rates"}))
    table_texts = get_table_texts(soup)
    if not has_rates or not table_texts:
        # Some pages have no price table
        campings[site]["flat"] = None
        campings[site]["adult"] = None
        campings[site]["child"] = None
    else:
        # Some use "Flat" for the flat rate, some use "Pitch"
        try:
            flat_price = get_price(table_texts, "Flat")
        except ValueError:
            flat_price = get_price(table_texts, "Pitch")

        adult_price = get_price(table_texts, "Adults")
        child_max_age, child_price = get_age_price(table_texts, "Children")

        # We don't know from the page how many persons are covered by the
        # flat rate, but we need some number; 2 is pretty common
        campings[site]["flat"] = {"price": flat_price, "persons_covered": 2}
        campings[site]["adult"] = {"price": adult_price, "age_min": child_max_age + 1}
        campings[site]["child"] = {"price": child_price, "age_max": child_max_age}

    # Find coordinates
    lat, lng = get_coords(soup)
    campings[site]["coords"] = (lat, lng)

### Functions for Calculating and Showing Summaries

In [8]:
def show_campingfrance_link(camping):
    link = HTML('Link to CampingFrance: <a href="{url}">click here</a>'.format(url=camping["href"]))
    display(link)


# Map shows car route from Mont-Dauphin
LOC_URL_FMT = (
    "https://graphhopper.com/maps/"
    "?point=Mont-Dauphin%2C%2005600%2C%20France"
    "&point={long}%2C{lat}"
    "&locale=en-US"
    "&vehicle=car"
    "&weighting=fastest"
    "&elevation=true"
    "&use_miles=false"
    "&layer=Omniscale"
)


def show_location_link(camping):
    long, lat = camping["coords"]
    loc_url = LOC_URL_FMT.format(long=long, lat=lat)
    location = HTML('Location: <a href="{url}">{long},{lat}</a>'.format(url=loc_url, long=long, lat=lat))
    display(location)


def show_prices_per_night(camping):
    if camping["flat"] is None:
        print(
            "Flat                               : N/A"
        )
    else:
        print(
            "Flat                               : {:.02f}".format(camping["flat"]["price"])
        )
    print(
        "Covers {} person(s) and {} car(s)".format(2, 1)  # no info, but good guess
    )

    if camping["adult"] is None:
        print(
            "Supplementary adult (age ?-)       : N/A"
        )
    else:
        print(
            "Supplementary adult (age {}-)       : {:.02f}".format(
                camping["adult"]["age_min"], camping["adult"]["price"]
            )
        )

    if camping["child"] is None:
        print(
            "Supplementary child (age 0-?)      : N/A"
        )
    else:
        print(
            "Supplementary child (age 0-{})      : {:.02f}".format(
                camping["child"]["age_max"], camping["child"]["price"]
            )
        )

        
def determine_covered_by_flat(camping):
    people_paid = set()
    for _ in range(N_PLACES):
        n_base_paid = 0
        for _ in range(camping["flat"]["persons_covered"]):
            adult = get_adult(PEOPLE, age_min=camping["adult"]["age_min"], exclude=people_paid)
            if adult is not None:
                assert adult not in people_paid
                people_paid.add(adult)
                n_base_paid += 1
        for _ in range(camping["flat"]["persons_covered"] - n_base_paid):
            child = get_child(
                PEOPLE, age_min=0, age_max=camping["child"]["age_max"], exclude=people_paid
            )
            if child is not None:
                assert child not in people_paid
                people_paid.add(child)
                n_base_paid += 1

    return people_paid


def calculate_totals(camping):
    # Flat price, covering some people already
    if camping["flat"] is None:
        flat_price = None
        people_paid = set()
    else:
        flat_price = camping["flat"]["price"] * N_PLACES * N_NIGHTS
        people_paid = determine_covered_by_flat(camping)

    # Price for supplementary adults and their number
    if camping["adult"] is None:
        n_supp_adults = 0
        supp_adults_price = None
    else:
        n_supp_adults = count_adults(
            PEOPLE, age_min=camping["adult"]["age_min"], exclude=people_paid
        )
        supp_adults_price = camping["adult"]["price"] * n_supp_adults * N_NIGHTS

    # Price for supplementary children and their number
    if camping["child"] is None:
        n_supp_children = 0
        supp_children_price = None
    else:
        n_supp_children = count_children(
            PEOPLE, age_min=0, age_max=camping["child"]["age_max"], exclude=people_paid
        )
        supp_children_price = camping["child"]["price"] * n_supp_children * N_NIGHTS

    return flat_price, (supp_adults_price, n_supp_adults), (supp_children_price, n_supp_children)


def show_totals(flat_price, supp_adults, supp_children):
    if flat_price is None:
        print("Flat                               : N/A")
    else:
        print("Flat                               : {:.02f}".format(flat_price))
    
    supp_adults_price, n_supp_adults = supp_adults
    if supp_adults_price is None:
        print("Supplementary adults               : N/A")
    else:
        print("Supplementary adults ({})           : {:.02f}".format(n_supp_adults, supp_adults_price))
        
    supp_children_price, n_supp_children = supp_children
    if supp_children_price is None:
        print("Supplementary children             : N/A")
    else:
        print("Supplementary children ({})         : {:.02f}".format(n_supp_children, supp_children_price))
    

### Print Summary

Unfortunately, many camping sites have no price information :-(

In [9]:
def summarize(camping):
    # Print summary
    print()
    print("=== {} ===".format(camping["name"]))
    print()
    
    show_campingfrance_link(camping)
    show_location_link(camping)
    print()
    print()

    print("--- Prices per Night ---")
    show_prices_per_night(camping)
    print()

    print("--- Calculated Total Prices ---")
    flat_price, supp_adults, supp_children = calculate_totals(camping)
    show_totals(flat_price, supp_adults, supp_children)
    print()

    supp_adults_price = supp_adults[0]
    supp_children_price = supp_children[0]
    total_prices = [flat_price, supp_adults_price, supp_children_price]
    if all(p is None for p in total_prices):
        total = None
    else:
        total = sum(p for p in total_prices if p is not None)
    if total is None:
        print("Total                              : N/A")
    else:
        print("Total                              : {:.02f}".format(total))
    print()
    

In [10]:
for site, camping in campings.items():
    summarize(camping)


=== Les Allouviers ===





--- Prices per Night ---
Flat                               : N/A
Covers 2 person(s) and 1 car(s)
Supplementary adult (age ?-)       : N/A
Supplementary child (age 0-?)      : N/A

--- Calculated Total Prices ---
Flat                               : N/A
Supplementary adults               : N/A
Supplementary children             : N/A

Total                              : N/A


=== Catinat Fleuri ===





--- Prices per Night ---
Flat                               : N/A
Covers 2 person(s) and 1 car(s)
Supplementary adult (age ?-)       : N/A
Supplementary child (age 0-?)      : N/A

--- Calculated Total Prices ---
Flat                               : N/A
Supplementary adults               : N/A
Supplementary children             : N/A

Total                              : N/A


=== La Place ===





--- Prices per Night ---
Flat                               : 7.80
Covers 2 person(s) and 1 car(s)
Supplementary adult (age 8-)       : 2.10
Supplementary child (age 0-7)      : 1.20

--- Calculated Total Prices ---
Flat                               : 109.20
Supplementary adults (1)           : 14.70
Supplementary children (2)         : 16.80

Total                              : 140.70


=== Les Melezes ===





--- Prices per Night ---
Flat                               : 7.20
Covers 2 person(s) and 1 car(s)
Supplementary adult (age 11-)       : 6.90
Supplementary child (age 0-10)      : 4.40

--- Calculated Total Prices ---
Flat                               : 100.80
Supplementary adults (1)           : 48.30
Supplementary children (2)         : 61.60

Total                              : 210.70


=== Les Milles Vents ===



...



--- Prices per Night ---
Flat                               : 5.50
Covers 2 person(s) and 1 car(s)
Supplementary adult (age 2-)       : 5.50
Supplementary child (age 0-1)      : 2.50

--- Calculated Total Prices ---
Flat                               : 77.00
Supplementary adults (3)           : 115.50
Supplementary children (0)         : 0.00

Total                              : 192.50


=== Le Verger ===





--- Prices per Night ---
Flat                               : N/A
Covers 2 person(s) and 1 car(s)
Supplementary adult (age ?-)       : N/A
Supplementary child (age 0-?)      : N/A

--- Calculated Total Prices ---
Flat                               : N/A
Supplementary adults               : N/A
Supplementary children             : N/A

Total                              : N/A


=== La Madeleine ===





--- Prices per Night ---
Flat                               : N/A
Covers 2 person(s) and 1 car(s)
Supplementary adult (age ?-)       : N/A
Supplementary child (age 0-?)      : N/A

--- Calculated Total Prices ---
Flat                               : N/A
Supplementary adults               : N/A
Supplementary children             : N/A

Total                              : N/A


=== Parc Le Villard ===





--- Prices per Night ---
Flat                               : 22.00
Covers 2 person(s) and 1 car(s)
Supplementary adult (age 8-)       : 6.00
Supplementary child (age 0-7)      : 5.00

--- Calculated Total Prices ---
Flat                               : 308.00
Supplementary adults (1)           : 42.00
Supplementary children (2)         : 70.00

Total                              : 420.00


=== Ferme De Civadille ===





--- Prices per Night ---
Flat                               : N/A
Covers 2 person(s) and 1 car(s)
Supplementary adult (age ?-)       : N/A
Supplementary child (age 0-?)      : N/A

--- Calculated Total Prices ---
Flat                               : N/A
Supplementary adults               : N/A
Supplementary children             : N/A

Total                              : N/A

